# Sampling Landscape Photos by using Generative Models

-------------

## Motivation

Da ich leidenschaftlich gerne fotografiere und sich meine Fotografie hauptsächlich auf die Landschaftsfotografie fokussiert, bin ich sehr interessiert ein generatives Modell zu erstellen, das selber Landschaften kreieren kann.

Impressionen: https://500px.com/p/visualframing?view=photos

In [ ]:
#!nvidia-smi
!pip install torchsummary

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
from torchvision.utils import make_grid
from torchsummary import summary
from torchvision import datasets

import matplotlib.pyplot as plt
import PIL
import os
import seaborn as sns
import numpy as np
from tqdm.notebook import tqdm
import pickle
import pandas as pd

In [ ]:
torch.manual_seed(0)

# Data

In [ ]:
N_IMAGES = 24
SEED = 11

np.random.seed(SEED)
random_images = np.random.choice(os.listdir('../input/impressionistlandscapespaintings/content/drive/MyDrive/impressionist_landscapes_resized_1024/'), 
                                 size=N_IMAGES, replace=False)
ncols = 3
nrows = N_IMAGES // ncols
fig = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 3*nrows))

for i, img in enumerate(random_images):
    plt.subplot(nrows, ncols, i+1)
    image = PIL.Image.open('../input/impressionistlandscapespaintings/content/drive/MyDrive/impressionist_landscapes_resized_1024/' + img)
    plt.imshow(image)
    plt.title('Name: ' + img, fontsize=8)
    plt.axis('off')
    
plt.show()

In [ ]:
plt.rcParams["savefig.bbox"] = 'tight'


def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(figsize=(12,8), ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])   
    plt.show()

In [ ]:
class CustomDataset(Dataset):
    """
    Class defines custom Dataset as a workaround to the ImageFolder class, which 
    did not return correct amount of batch size.
    """

    def __init__(self, root_dir: str, transform: 'Compose' = None, dataset_fraction: float = 1):
        """
        
        Params:
        ------------------
        root_dir: str
            Defines the path from where all images should be imported from
        
        transform: torch.utils.transforms.Compose
            Compose of different transforms applied during import of an image.
            
        all_images: list
            List of all images names in the root dir.
        
        dataset_fraction: int = 1
            fraction of dataset to take from for training. default = 1
        """
        self.root_dir = root_dir
        self.transform = transform
        self.dataset_fraction = dataset_fraction
        self.images = self._build_image_list()
        
    def _build_image_list(self):
        # Calculate fraction of data to take from dataset
        dataset_size = int(round(len(os.listdir(self.root_dir)) * self.dataset_fraction, 0))
        # Load all images from within root dri
        all_images = np.array([img for img in os.listdir(self.root_dir) if '.jpg' in img])
        
        return np.random.permutation(all_images)[:dataset_size]
    
    def get_len_root(self):
        return len(os.listdir(self.root_dir))
        
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = PIL.Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image
    
def get_dataloader(root_dir: str, transforms: 'torch.utils.Compose', 
                   batch_size: int, workers: int, dataset_fraction: float = 1):
    """
    Functino returns a dataloader with given parameters
    
    Params:
    ---------------
    root_dir: str
        Defines the path from where all images should be imported from
        
    transform: torch.utils.transforms.Compose
        Compose of different transforms applied during import of an image.
        
    batch_size; int
        Size of the imported batch
        
    woerkers: int
        Amount of CPU workers for the loading of data into gpu.
    """
    
    custom_dataset = CustomDataset(root_dir=root_dir, transform=transforms, dataset_fraction=dataset_fraction)
    
    return DataLoader(dataset=custom_dataset, 
                      batch_size=batch_size, 
                      num_workers=workers, 
                      shuffle=True)
    

## Model

In [ ]:
# According to Implementation https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
class DCGANDiscriminator(nn.Module):
    def __init__(self):
        super(DCGANDiscriminator, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(64 * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)
    
# Generator Code

class DCGANGenerator(nn.Module):
    def __init__(self):
        super(DCGANGenerator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( 100, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( 64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( 64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( 64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
  
# According to Implementation https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
class DCGANDiscriminator128(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # input is (nc) x 64 x 64
            nn.Conv2d(64, 64*2, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(64 * 8, 64 * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 16),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(64 * 16, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)
    
# Generator Code

class DCGANGenerator128(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( 100, 64 * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 16),
            nn.Dropout2d(p=.5, inplace=True),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(64 * 16, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.Dropout2d(p=.5, inplace=True),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( 64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.Dropout2d(p=.5, inplace=True),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( 64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.Dropout2d(p=.5, inplace=True),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( 64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 1),
            nn.Dropout2d(p=.5, inplace=True),
            nn.ReLU(True),
            # state size. (nc) x 64 x 64
            nn.ConvTranspose2d( 64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
            # out 3x128x128
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
summary(DCGANDiscriminator128().to('cuda'), (3, 128, 128))

In [ ]:
summary(DCGANGenerator128().to('cuda'), (100, 1, 1))

## Dataloader

In [ ]:
# Params 
BATCH_SIZE = 32
WORKERS = 2
IMAGE_ROOT = '../input/impressionistlandscapespaintings/content/drive/MyDrive/impressionist_landscapes_resized_1024'

In [ ]:
dataloader = get_dataloader(root_dir=IMAGE_ROOT, 
                            transforms=transforms.Compose([transforms.Resize(size=64),
                                                           transforms.ToTensor()]),
                            batch_size=BATCH_SIZE, workers=WORKERS)

In [ ]:
real_batch = next(iter(dataloader))

In [ ]:
show(make_grid(real_batch))

In [ ]:
assert len(real_batch) == BATCH_SIZE, f'{len(real_batch)}, {BATCH_SIZE}'
print('--- Test Passed ---')
del real_batch

In [ ]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print(device)

# DCGAN - Deep Convolutional Generative Adversial Network


In [ ]:
z_vec = torch.rand(BATCH_SIZE,100,1,1)
print('Z-Vector:', z_vec.shape)

In [ ]:
# Test Generator
generator = DCGANGenerator128()
out = generator(z_vec)
print(out.shape)
assert out.shape == torch.Size([BATCH_SIZE, 3, 128, 128])
print('--- Test Passed ---')
show(make_grid(out))

In [ ]:
# Test Discrimnator
discriminator = DCGANDiscriminator128()
sample_image = torch.rand((3, 128, 128))
sample_image = sample_image.unsqueeze(0)

out = discriminator(sample_image)
print(out.shape)
assert out.shape == torch.Size([1, 1, 1, 1])
print('--- Test Passed ---')

## Model Training

In [ ]:
def train_network(generator, discriminator, criterion, 
                  optimizer_generator, optimizer_discriminator, 
                  n_epochs, dataloader, z_dim, inverse_transforms=None, 
                  epsilon=None, debug_run=False, show_images=True, label_smoothing = False,
                  **kwargs):
    """
    Trains GAN by using batch-wise fake and real data inputs.
    
    params:
    --------------------
    generator: torch.Network
        Generator model to sample image from latent space.
        
    discriminator: torch.Network
        Discrimantor Model to classify into real or fake.
        
    criterion: 
        Cost-Function used for the network optimizatio
        
    optimizer: torch.Optimizer
        Optmizer for the network
        
    n_epochs: int
        Defines how many times the whole dateset should be fed through the network
        
    dataloader: torch.Dataloader 
        Dataloader with the batched dataset of real data.
        
    epsilon: float
        Stopping Criterion regarding to change in cost-function between two epochs.
        
    debug_run:
        If true than only one batch will be put through network.
        
    returns:
    ---------------------
    generator:
        Trained Torch Generator Model
        
    discriminator:
        Trained Torch Discriminator Model
        
    losses: dict
        dictionary of losses of all batches and Epochs.
        
    """
    print(20*'=', 'Start Training', 20*'=')
    # Init lists to keep track of loss
    training_loss_generator, training_loss_discriminator = [], []
    batch_loss = {'Dreal':[], 'Dfake':[], 'Gfake':[]}
    accuracy = {'real':[], 'fake':[], 'Gfake':[]}
    # Accuracy func
    calculate_accuracy = lambda y_true, y_pred: np.sum(y_true == y_pred) / y_true.shape[0]
    
    # Fixed noise for generating samples
    fixed_noise_dim = kwargs.get('fixed_noise_dim') if kwargs.get('fixed_noise_dim') else dataloader.batch_size
    fixed_noise = torch.randn(fixed_noise_dim, z_dim, 1, 1, device=device)
    
    dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    print(f'Training on: {dev}')
    generator.to(dev), discriminator.to(dev)
    criterion.to(dev)

    generator.train(), discriminator.train()
    overall_length = len(dataloader)
    with tqdm(total=n_epochs*overall_length, disable=debug_run) as pbar:
        for epoch in range(n_epochs):  # loop over the dataset multiple times
            discriminator_running_loss, generator_running_loss = 0.0, 0.0
            for i, data in enumerate(dataloader): 
                
                # Get Batch of real images and pass to Discriminator which first trains on only real data.
                real_images = data.to(dev)
                
                # Create Labels
                real_labels = torch.full(size=(real_images.shape[0], ), fill_value=real_label, 
                                         dtype=torch.float, device=device)
                fake_labels = torch.full(size=(real_images.shape[0], ), fill_value=fake_label, 
                                         dtype=torch.float, device=device)
                flipped_fake_labels = torch.full(size=(real_images.shape[0], ), fill_value=real_label, 
                                                 dtype=torch.float, device=device)  
                if label_smoothing:
                    real_labels += torch.normal(mean=0, std=.1, size=(real_labels.shape[0], ), device=dev)
                    fake_labels += torch.abs(torch.normal(mean=0, std=.1, size=(real_labels.shape[0], ), device=dev))
                    flipped_fake_labels += torch.normal(mean=0, std=.1, size=(real_labels.shape[0], ), device=dev)
                
                
                # Create noise from normal distribution
                # Generate Batch of latent vectors
                noise_vec = torch.randn(real_images.shape[0], Z_DIM, 1, 1, device=device)                
                
                # -----------------------------------------
                # REAL IMAGES - Training Discriminator 
                # -----------------------------------------
                discriminator.zero_grad()
                
                # Pass into Discriminator
                out = discriminator(real_images).view(-1)
                loss_real = criterion(out, real_labels)
                loss_real.backward()
                
                batch_loss['Dreal'].append(loss_real.mean().item())
                pred = torch.round(out)
                accuracy['real'].append(calculate_accuracy(real_labels.cpu().numpy(), 
                                                           pred.detach().cpu().numpy()))  
                 
                # -----------------------------------------
                # Fake Images - Discriminator
                # -----------------------------------------
                
                # Pass noise to generator
                fake_images = generator(noise_vec)
                # Passed generated images to discriminator
                out = discriminator(fake_images.detach()).view(-1)
                
                # Calculate Loss on Fake images
                loss_fake = criterion(out, fake_labels)
                loss_fake.backward()
                
                d_loss = loss_real + loss_fake
                
                # Udpate Discriminator Optimizer
                optimizer_discriminator.step()
                
                batch_loss['Dfake'].append(loss_fake.mean().item())
                
                # Calculate overall loss over both and append
                discriminator_running_loss += d_loss.mean().item()
                
                # Calculate accuracy on real images 
                pred = torch.round(out)
                accuracy['fake'].append(calculate_accuracy(fake_labels.cpu().numpy(), 
                                                           pred.detach().cpu().numpy()))
                # -----------------------------------------
                # FAKE - Training of Generator 
                # -----------------------------------------
                generator.zero_grad()
                                
                #fake_images = generator(G_z_vec)
                out = discriminator(fake_images).view(-1)
                loss_generator = criterion(out, flipped_fake_labels)
                loss_generator.backward()
                
                # Update G
                optimizer_generator.step()
                
                # Append Losses 
                batch_loss['Gfake'].append(loss_generator.mean().item())
                pred = torch.round(out)
                accuracy['Gfake'].append(calculate_accuracy(1-flipped_fake_labels.cpu().numpy(), 
                                                            pred.detach().cpu().numpy()))
                generator_running_loss += loss_generator.mean().item()
                
                # ----------------------------------------
                # calc and print stats
                pbar.set_description(f'Epoch: {epoch+1}/{n_epochs} // GRL: {round(generator_running_loss, 3)}  -- '+
                                     f'DRL: {round(discriminator_running_loss, 3)} ')
                pbar.update(1)
                if debug_run:
                    print('- Training Iteration passed. -')
                    break
                
            print(f'Epoch {epoch+1} // [GRL: {np.round(generator_running_loss, 3)}]  -- '+
                  f'[DRL: {np.round(discriminator_running_loss, 3)}] -- ' + 
                  f'[Accuracy (R/F): {round(np.mean(accuracy["real"][:-i]), 3)} - ',
                  f'{round(np.mean(accuracy["fake"][:-i]), 3)}]')
            
            training_loss_generator.append(generator_running_loss)
            training_loss_discriminator.append(discriminator_running_loss)
            
            if show_images:
                with torch.no_grad():
                    imgs = generator(fixed_noise).detach().cpu()
                    if inverse_transforms:
                        imgs = inverse_transforms(imgs)
                    show(make_grid(imgs))
                    
            
            if epsilon:
                if epoch > 0:
                    diff = np.abs(training_loss_generator[-2] - generator_running_loss)
                    if diff < epsilon:
                        print('- Network Converged. Stopped Training. -')
                        break

            if debug_run:
                # Breaks loop 
                break
        
    print(20*'=', 'Finished Training', 20*'=')
    return generator, discriminator, dict(generator=training_loss_generator,
                                          discriminator=training_loss_discriminator,
                                          batch_loss=batch_loss, accuracy=accuracy)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0)

In [ ]:
# Init Networks
discriminator = DCGANDiscriminator128()
discriminator.apply(weights_init)

generator = DCGANGenerator128()
generator.apply(weights_init)

torch.cuda.empty_cache()

####################################################
# Training Parameters
Z_DIM = 100
IMAGE_SIZE = 128
N_EPOCHS = 100
DEBUG_RUN = False
IMAGE_ROOT = '../input/impressionistlandscapespaintings/content/drive/MyDrive/impressionist_landscapes_resized_1024'
WORKERS = 0 
BATCH_SIZE = 64 
DS_FRACTION = 1 # Will only take a fraction of the dataset
fixed_noise_dim = 32
####################################################

dataloader = get_dataloader(root_dir=IMAGE_ROOT, 
                            transforms=transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                                           transforms.ToTensor(), 
                                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
                            batch_size=BATCH_SIZE, workers=WORKERS, dataset_fraction=DS_FRACTION)

"""dataloader = get_dataloader(root_dir=IMAGE_ROOT, 
                            transforms=transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                                           transforms.ToTensor()]),
                            batch_size=BATCH_SIZE, workers=WORKERS, dataset_fraction=DS_FRACTION)"""

# Defines label to train on 
real_label = 1.
fake_label = 0.

criterion = nn.BCELoss()
optimizer_generator = optim.Adam(generator.parameters(), lr=.0002, betas=(.5, .999))
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=.0002, betas=(.5, .999))

inverse_transforms = transforms.Compose([transforms.Normalize((0, 0, 0), (1/.5, 1/.5, 1/.5)), 
                                         transforms.Normalize((-0.5, -0.5, -0.5), (1, 1, 1))])

In [ ]:
generator, discriminator, loss = train_network(generator=generator, discriminator=discriminator, 
                                               criterion=criterion, optimizer_generator=optimizer_generator, 
                                               optimizer_discriminator=optimizer_discriminator, z_dim=Z_DIM,
                                               n_epochs=N_EPOCHS, dataloader=dataloader, debug_run=DEBUG_RUN, 
                                               fixed_noise_dim=fixed_noise_dim, label_smoothing=False, 
                                               inverse_transforms = inverse_transforms)

In [ ]:
def plot_batch_loss(loss: dict):
    tmp = pd.DataFrame.from_dict(loss['batch_loss']) \
        .reset_index() \
        .melt(id_vars='index', value_name='loss', var_name='type')
    
    fig = plt.subplots(figsize=(8, 4))
    p = sns.lineplot(x=tmp['index'], y=tmp['loss'], style=tmp['type'], 
                     hue=tmp['type'], palette='Paired')
    p.set_title('Batch Loss', loc='left')
    p.set_xlabel('Batch')
    p.set_ylabel('BCE-Loss')
    sns.despine()
    plt.show()
    
def plot_batch_acc(loss: dict):
    tmp = pd.DataFrame.from_dict(loss['accuracy']) \
        .reset_index() \
        .melt(id_vars='index', value_name='acc', var_name='type')
    
    fig = plt.subplots(figsize=(8, 4))
    p = sns.lineplot(x=tmp['index'], y=tmp['acc'], style=tmp['type'], 
                     hue=tmp['type'], palette='Paired')
    p.set_title('Batch Accuracy', loc='left')
    p.set_xlabel('Batch')
    p.set_ylabel('Accuracy')
    sns.despine()
    plt.show()

In [ ]:
plot_batch_loss(loss)

In [ ]:
plot_batch_acc(loss)

In [ ]:
# Save states
SAVE_OUTPUT = True

if SAVE_OUTPUT:
    with open('./losses_s128_12eps.pkl', 'wb') as pkl_file:
        pickle.dump(loss, pkl_file)
    
    torch.save(generator, f'./generator_{N_EPOCHS}eps_{IMAGE_SIZE}pix')
    torch.save(discriminator, f'./discriminator_{N_EPOCHS}eps_{IMAGE_SIZE}pix')

In [ ]:
def plot_image_batch(Z):
    """"""
    ncols = 6
    nrows = Z.shape[0] // ncols + 1
    
    with torch.no_grad():
        image_batch = generator(Z)
    
    inverse_transforms = transforms.Compose([transforms.Normalize(mean=(0, 0, 0), std=(1/0.5, 1/0.5, 1/0.5)),
                                             transforms.Normalize(mean=(-.5, -.5, -.5), std=(1, 1, 1)),
                                             transforms.ToPILImage()])     
    fig = plt.subplots(figsize=(16, nrows*3))
    
    for i, image in enumerate(image_batch):
        plt.subplot(nrows, ncols, i+1)
        plt.imshow(inverse_transforms(image))
        plt.axis('off')
    plt.show()

In [ ]:
plot_image_batch(Z=loss['generated_images'][-1])

## Generate Images

generator = torch.load('./models/generator_12eps')

In [ ]:
with torch.no_grad():
    out = generator(torch.randn(1, Z_DIM, 1, 1, device=device))
    
inverse_transforms = transforms.Compose([transforms.ToPILImage()]) 
image = inverse_transforms(out[0].to('cpu'))
plt.imshow(image)

In [ ]:
out[0].shape

torch.save(discriminator, 'discriminator_4eps')

# Sources

https://paperswithcode.com/dataset/lhq

https://universome.github.io/alis

https://arxiv.org/pdf/1511.06434.pdf

https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html

https://github.com/soumith/ganhacks